In [3]:
#Prepare Training Data for Generator (Conditioned on Mood)
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [4]:
# === CONFIG ===
PICKLE_PATH = "D:/Study/Ai_Music_Composer/project/data/note_sequences.pkl"
SEQ_LEN = 50  # Sequence length used during extraction

In [5]:
# === LOAD SEQUENCES ===
with open(PICKLE_PATH, 'rb') as f:
    data = pickle.load(f)

note_sequences = data['sequences']
labels = data['labels']

In [6]:
# === BUILD_NOTE VOCABULARY ===
unique_notes = sorted(set(note for seq in note_sequences for note in seq))
note_to_idx = {note: i for i, note in enumerate(unique_notes)}
idx_to_note = {i: note for note, i in note_to_idx.items()}
vocab_size = len(note_to_idx)

with open("D:\Study\Ai_Music_Composer\project\data\generator_mappings.pkl", "wb") as f:
    pickle.dump({
        "note_to_idx": note_to_idx,
        "idx_to_note": idx_to_note,
        "seq_len": SEQ_LEN
    }, f)

In [7]:
# === ENCODE MOOD LABELS ===
mood_encoder = LabelEncoder()
mood_labels = mood_encoder.fit_transform(labels)
num_moods = len(mood_encoder.classes_)

In [8]:
# === BUILD INPUTS AND TARGETS ===
MAX_SAMPLES = 100000

X_notes = []
X_moods = []
y = []

count = 0

for i in range(len(note_sequences)):
    seq = note_sequences[i]
    mood = mood_labels[i]

    for j in range(1, len(seq)):
        input_seq = seq[:j]
        target_note = seq[j]

        # Pad with 0s if shorter than SEQ_LEN
        input_seq = [0] * (SEQ_LEN - len(input_seq)) + input_seq[-SEQ_LEN:]

        try:
            X_notes.append([note_to_idx[n] for n in input_seq])
            X_moods.append(mood)
            y.append(note_to_idx[target_note])
        except KeyError:
            continue

        count += 1
        if count >= MAX_SAMPLES:
            break

    if count >= MAX_SAMPLES:
        break

print(f"✅ Prepared {len(X_notes)} training samples.")

✅ Prepared 100000 training samples.


In [9]:
# === CONVERT TO NUMPY ARRAYS ===
X_notes = np.array(X_notes)
X_moods = to_categorical(np.array(X_moods), num_classes=num_moods)
y = to_categorical(np.array(y), num_classes=vocab_size)

In [10]:
# === SAVE DATA FOR TRAINING ===
with open("D:/Study/Ai_Music_Composer/project/data/generator_training_data.pkl", "wb") as f:
    pickle.dump({
        "X_notes": X_notes,
        "X_moods": X_moods,
        "y": y,
        "note_to_idx": note_to_idx,
        "idx_to_note": idx_to_note,
        "mood_encoder": mood_encoder.classes_.tolist()
    }, f)

print("✅ Generator training data prepared and saved.")

✅ Generator training data prepared and saved.
